## Architecture code

### CNN Architecture

In [1]:
# -------------------------------------------------------------------------------------------------------------
# IMPORTS
# -------------------------------------------------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as functional
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms


#### Test data

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [7]:
train_data = torchvision.datasets.MNIST(root='../data', train=True, transform=transform, download=True)
test_data = torchvision.datasets.MNIST(root='../data', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:15<00:00, 636kB/s] 


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 259kB/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:03<00:00, 428kB/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 8.92MB/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



#### Branch 1

In [17]:
# -------------------------------------------------------------------------------------------------------------
# @CLASS: CNN_ARCHITECTURE
# -------------------------------------------------------------------------------------------------------------
class CNN_ARCHITECTURE(nn.Module):
    def __init__(self, input_size, hidden_layers, activation, norm_layer, drop_prob=0.0):
        super(CNN_ARCHITECTURE, self).__init__()

        # Initialize variables
        self.input_size = input_size
        self.hidden_layers = hidden_layers
        self.activation = activation
        self.norm_layer = norm_layer
        self.drop_prob = drop_prob

        self.build_model()

    """-------------------------------------------------------------------------------------------------------------
    @Function: build_model
    @Args: self
    @Returns: features
    @Description: Create a CNN architecture that extracts the features from the input images
    -------------------------------------------------------------------------------------------------------------"""
    def build_model(self):

        layers = []
        input_dim = self.input_size
        for i in range(len(self.hidden_layers)):
            # Add the convoluiton layer
            layers.append(nn.Conv2d(input_dim, self.hidden_layers[i], 3, stride=1, padding=1))

            # Add batch normalization
            if self.norm_layer:
                layers.append(self.norm_layer(self.hidden_layers[i]))
            
            # Add maxpooling, change this condition
            if True:
                layers.append(nn.MaxPool2d(2,2))
            
            # Add activation
            layers.append(self.activation())

            # Add dropout
            if self.drop_prob:
                layers.append(nn.Dropout(self.drop_prob))

            input_dim = self.hidden_layers[i]
        
        self.features = nn.Sequential(*layers)

    """-------------------------------------------------------------------------------------------------------------
    @Function: forward
    @Args: 
        self: object
        x: inputs
    @Returns: features
    @Description: Passes the input to the CNN architecture for feature extraction
    -------------------------------------------------------------------------------------------------------------"""
    def forward(self, x):
        features = self.features(x)
        features = features.view(features.size(0), -1)
        return features
    

    """-------------------------------------------------------------------------------------------------------------
    @Function: image_denoising
    @Args: 
        self: object
    @Returns: denoised images
    @Description: A denoising filter to remove the noise from the input images
    -------------------------------------------------------------------------------------------------------------"""
    def image_denoising(self):
        print("I am denoising")

In [18]:
cnn_experiment_1 = dict(
    name = 'CNN_Experiment_1',

    model_args = dict(
        input_size = 3,
        num_classes = 10,
        hidden_layers = [128, 512, 512],
        activation = nn.ReLU,
        norm_layer = False,
        drop_prob = 0.4,
    ),
)

# from configs.cnn_branch_config import cnn_experiment_1
print(cnn_experiment_1['model_args']['hidden_layers'])

[128, 512, 512]


In [19]:
# -------------------------------------------------------------------------------------------------------------
# DRIVER CODE
# -------------------------------------------------------------------------------------------------------------
# from configs.cnn_branch_config import cnn_experiment_1

model = CNN_ARCHITECTURE(3, cnn_experiment_1['model_args']['hidden_layers'], cnn_experiment_1['model_args']['activation'], cnn_experiment_1['model_args']['norm_layer'], cnn_experiment_1['model_args']['drop_prob'])
model.eval()

input = torch.randn(1,3,224,224)

with torch.no_grad():
    final_features = model(input)

print(final_features)
print(final_features.shape)

tensor([[0.0856, 0.0000, 0.0000,  ..., 0.0202, 0.0000, 0.1317]])
torch.Size([1, 401408])
